In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import OneHotEncoder

In [2]:
np.random.seed(1)

In [3]:
df = pd.read_csv("anemia.csv")
df.head()

,ID,center,age,sex,BMI,type2,duration,HTN,HLP,vascinc,...,days_dialysis,RRT,KTx,SCr_fup,days_SCr_fup,crea,SCr_diff,SCr_double,progression,days_progression
0,M003,1,56.065753,1,26.297578,1,1.0,0,0,0,...,1545,0,0,0.84,2204.0,0.87,-0.03,0.0,0.0,2204.0
1,M008,1,55.339726,1,33.208550,1,0.5,1,1,0,...,1506,0,0,1.00,2.0,0.98,0.02,0.0,0.0,2.0
2,M009,1,75.786301,0,27.688778,1,6.0,1,0,1,...,1539,0,0,0.82,1.0,0.81,0.01,0.0,0.0,1.0
3,M015,1,43.213699,0,NaN,1,1.0,0,0,0,...,2197,0,0,0.73,1981.0,0.83,-0.10,0.0,0.0,1981.0
4,M033,1,60.060274,0,26.927438,1,4.0,1,1,1,...,1696,0,0,0.94,1558.0,0.90,0.04,0.0,0.0,1558.0


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 249 entries, 0 to 248
Data columns (total 43 columns):
ID                  249 non-null object
center              249 non-null int64
age                 249 non-null float64
sex                 249 non-null int64
BMI                 246 non-null float64
type2               249 non-null int64
duration            240 non-null float64
HTN                 249 non-null int64
HLP                 249 non-null int64
vascinc             249 non-null int64
ACE                 248 non-null float64
ARB                 248 non-null float64
statin              248 non-null float64
hb                  240 non-null float64
CRP                 232 non-null float64
alb                 235 non-null float64
HbA1c               238 non-null float64
ferritin            235 non-null float64
EPO                 242 non-null float64
chol                230 non-null float64
LDL                 228 non-null float64
HDL                 229 non-null float64
hepcid

## Data Cleaning

In [5]:
target = 'death'

### Replace empty or errors by np.nan

In [6]:
# replace empty/erros by np.nan
df = df.replace(r'^\s+$', np.nan, regex=True)
df = df.replace('\t','',regex=True)
df = df.replace(' ','',regex=True)
df = df.replace('\?',np.nan,regex=True)
df = df.replace('\<',np.nan,regex=True)
df = df.replace('#NULL!',np.nan,regex=True)
#df = df.replace('99',np.nan,regex=True)
#df = df.replace(99,np.nan,regex=True)
#df

### Remove outliers

In [7]:
# Drop the column outliers as it is unrelated to the dependent variable
outliers = ['ID','cause_of_death']
df = df.drop(outliers,axis=1)
df.head()

,center,age,sex,BMI,type2,duration,HTN,HLP,vascinc,ACE,...,days_dialysis,RRT,KTx,SCr_fup,days_SCr_fup,crea,SCr_diff,SCr_double,progression,days_progression
0,1,56.065753,1,26.297578,1,1.0,0,0,0,0.0,...,1545,0,0,0.84,2204.0,0.87,-0.03,0.0,0.0,2204.0
1,1,55.339726,1,33.208550,1,0.5,1,1,0,1.0,...,1506,0,0,1.00,2.0,0.98,0.02,0.0,0.0,2.0
2,1,75.786301,0,27.688778,1,6.0,1,0,1,1.0,...,1539,0,0,0.82,1.0,0.81,0.01,0.0,0.0,1.0
3,1,43.213699,0,NaN,1,1.0,0,0,0,NaN,...,2197,0,0,0.73,1981.0,0.83,-0.10,0.0,0.0,1981.0
4,1,60.060274,0,26.927438,1,4.0,1,1,1,0.0,...,1696,0,0,0.94,1558.0,0.90,0.04,0.0,0.0,1558.0


### Remove bad columns

In [8]:
def remove_bad_columns(df,bad_column_threshold):
    # find bad columns having too many missing values
    n_null = np.array(df.isnull().sum(axis=0))
    bad_col = np.array([]).astype(int)
    for i in range(len(n_null)):
        if n_null[i] >= bad_column_threshold:
            bad_col = np.append(bad_col,i)

    #print(bad_col)
    print('number of bad columns:',len(bad_col))

    # delete bad columns
    df = df.drop(df.columns[bad_col],axis=1)
    #df.info()
    return df  

In [9]:
df = remove_bad_columns(df,41)
df.info()

number of bad columns: 2
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 249 entries, 0 to 248
Data columns (total 39 columns):
center              249 non-null int64
age                 249 non-null float64
sex                 249 non-null int64
BMI                 246 non-null float64
type2               249 non-null int64
duration            240 non-null float64
HTN                 249 non-null int64
HLP                 249 non-null int64
vascinc             249 non-null int64
ACE                 248 non-null float64
ARB                 248 non-null float64
statin              248 non-null float64
hb                  240 non-null float64
CRP                 232 non-null float64
alb                 235 non-null float64
HbA1c               238 non-null float64
ferritin            235 non-null float64
EPO                 242 non-null float64
chol                230 non-null float64
LDL                 228 non-null float64
HDL                 229 non-null float64
hepcidin_ngml       2

### Remove bad rows

In [10]:
 # Find rows where target is missing
def find_missing_target_rows(df,target):
    # find rows where target is missing
    missing_row = df[target].isnull()
    print('Number of rows where target are missing:')
    print(sum(missing_row))

    #df = df[~missing_row]
    missing_row_indices = np.array([t for t in range(df.shape[0]) if missing_row[t]])
    
    return missing_row_indices

In [11]:
missing_target_rows = find_missing_target_rows(df,target)

Number of rows where target are missing:
0


We find bad rows which contain too many missing values, then remove them.

In [12]:
def find_bad_rows(df,bad_row_threshold):   
    # find bad rows having too many missing values
    n_null = np.array(df.isnull().sum(axis=1))
    bad_row = np.array([]).astype(int)
    for t in range(len(n_null)):
        if n_null[t] >= bad_row_threshold:
            bad_row = np.append(bad_row,t)

    #print(bad_row)
    print('number of bad rows:',len(bad_row))

    # delete bad rows
    #df = df.drop(bad_row)
    #df.info()
    return bad_row

In [13]:
bad_rows = find_bad_rows(df,6)
df.info()

number of bad rows: 13
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 249 entries, 0 to 248
Data columns (total 39 columns):
center              249 non-null int64
age                 249 non-null float64
sex                 249 non-null int64
BMI                 246 non-null float64
type2               249 non-null int64
duration            240 non-null float64
HTN                 249 non-null int64
HLP                 249 non-null int64
vascinc             249 non-null int64
ACE                 248 non-null float64
ARB                 248 non-null float64
statin              248 non-null float64
hb                  240 non-null float64
CRP                 232 non-null float64
alb                 235 non-null float64
HbA1c               238 non-null float64
ferritin            235 non-null float64
EPO                 242 non-null float64
chol                230 non-null float64
LDL                 228 non-null float64
HDL                 229 non-null float64
hepcidin_ngml       249

In [14]:
del_rows = np.union1d(missing_target_rows,bad_rows)
print('number of rows need to delete:',len(del_rows))

df = df.drop(del_rows)

number of rows need to delete: 13


### Separate target and attributes

In [15]:
dfx = df.drop(target,axis=1)
dfy = df[target]

### Find variable type

In [16]:
# number of uniques of each column (excluding NaN)
nu = np.array([len(pd.unique(dfx[col].dropna())) for col in dfx.columns])
print('number of uniques of each variable:')
print(nu)

number of uniques of each variable:
[  4 231   2 226   2  42   2   2   2   2   2   2  79 116 122  55 175 153
 124 110  59 124 236 236 184  98 210   2 206   3   2 114 195 111  99   2
   2 214]


In [17]:
def define_variable_type(df,nu):
    i_binary = [] ; i_category = [] ; i_continuous = []
    for i in range(len(nu)):
        if nu[i] == 2: # binary 
            i_binary.append(i)
        elif nu[i] < 5: # !!!! NOTE: this is not always correct, depending on data
            i_category.append(i)
        else:
            i_continuous.append(i)

    print('i_binary:',i_binary)
    print('i_category:',i_category)   
    #i_binary, i_category, i_continuous
    
    variable_type  = np.ones(len(nu))  # binary
    variable_type[i_category] = 2   # categorical
    variable_type[i_continuous] = 3 # continuous

    return variable_type #,i_binary,i_category,i_continuous

In [18]:
variable_type = define_variable_type(dfx,nu)
print('variable type:',variable_type)

i_binary: [2, 4, 6, 7, 8, 9, 10, 11, 27, 30, 35, 36]
i_category: [0, 29]
variable type: [2. 3. 1. 3. 1. 3. 1. 1. 1. 1. 1. 1. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3.
 3. 3. 3. 1. 3. 2. 1. 3. 3. 3. 3. 1. 1. 3.]


### Impute missing values of attributes

In [19]:
def impute_missing(df,variable_type):
    # impute binary and categorical variables by the most frequency (in each column)
    # continuous variable by median
    # input: df: pandas data frame, variable_type: list
    # output: df2: pandas data frame
    df2 = df.copy()
    for i,col in enumerate(df.columns):
        if variable_type[i] < 3: # binary or caterogy
            df2[col] = df[col].fillna(df[col].mode().iloc[0])
        else: # continuous
            df2[col] = df[col].fillna(df[col].median())    
    return df2       

In [20]:
dfx_imputed = impute_missing(dfx,variable_type)
dfx_imputed.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 236 entries, 0 to 248
Data columns (total 38 columns):
center              236 non-null int64
age                 236 non-null float64
sex                 236 non-null int64
BMI                 236 non-null float64
type2               236 non-null int64
duration            236 non-null float64
HTN                 236 non-null int64
HLP                 236 non-null int64
vascinc             236 non-null int64
ACE                 236 non-null float64
ARB                 236 non-null float64
statin              236 non-null float64
hb                  236 non-null float64
CRP                 236 non-null float64
alb                 236 non-null float64
HbA1c               236 non-null float64
ferritin            236 non-null float64
EPO                 236 non-null float64
chol                236 non-null float64
LDL                 236 non-null float64
HDL                 236 non-null float64
hepcidin_ngml       236 non-null float64
CKDEP

## Data Processing

### Attributes

In [21]:
def convert_binary_and_category(x,variable_type):
    """
    convert binary to +-1, category to one hot; remain continuous.
    """    
    onehot_encoder = OneHotEncoder(sparse=False,categories='auto')

    # create 2 initial columns
    x_new = np.zeros((x.shape[0],2))

    for i,i_type in enumerate(variable_type):
        if i_type == 1: # binary
            unique_value = np.unique(x[:,i])
            x1 = np.array([-1. if value == unique_value[0] else 1. for value in x[:,i]])        
            x_new = np.hstack((x_new,x1[:,np.newaxis]))

        elif i_type == 2: # category
            x1 = onehot_encoder.fit_transform(x[:,i].reshape(-1,1))
            x_new = np.hstack((x_new,x1))
            
        else: # continuous      
            x_new = np.hstack((x_new,x[:,i][:,np.newaxis]))      

    # drop the 2 initial column
    x_new = x_new[:,2:]
    
    return x_new.astype(float)

In [22]:
# convert x
x = np.array(dfx_imputed)
x_new = convert_binary_and_category(x,variable_type)

print(x_new.shape)
print(x_new)

(236, 43)
[[ 1.000e+00  0.000e+00  0.000e+00 ... -1.000e+00 -1.000e+00  2.204e+03]
 [ 1.000e+00  0.000e+00  0.000e+00 ... -1.000e+00 -1.000e+00  2.000e+00]
 [ 1.000e+00  0.000e+00  0.000e+00 ... -1.000e+00 -1.000e+00  1.000e+00]
 ...
 [ 1.000e+00  0.000e+00  0.000e+00 ... -1.000e+00  1.000e+00  7.600e+01]
 [ 1.000e+00  0.000e+00  0.000e+00 ... -1.000e+00  1.000e+00  4.660e+02]
 [ 0.000e+00  1.000e+00  0.000e+00 ... -1.000e+00 -1.000e+00  1.694e+03]]


### Target

In [23]:
y = np.array(dfy)
#print(np.unique(y,return_counts=True))

# convert taget to 0 and 1
y_new = y
#y_new = np.ones(y.shape[0])
#y_new[y =='No'] = 0

print(np.unique(y_new,return_counts=True))

(array([0, 1]), array([193,  43]))


In [24]:
# combine X and y and save to a file
xy_new = np.hstack((x_new,y_new[:,np.newaxis]))
np.savetxt('data_processed.dat',xy_new,fmt='%f')